# Textklassifikation am Beispiel [GermEval 2021](https://germeval2021toxic.github.io/SharedTask/)

Dieses Notebook ist eine adaptierte Version des Beispielskripts [examples/pytorch/text-classification/run_glue.py](https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py) von Huggingface.


In [1]:
#!/usr/bin/env python
# coding=utf-8
# Copyright 2020 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
!pip install wandb transformers datasets
!wandb login aee0ebe6843a7b138378e5ace3fdad105494724f

!test -e GermEval21_Toxic_Train.csv || wget -O GermEval21_Toxic_Train.csv https://raw.githubusercontent.com/cgawron/GermEval_2021/main/GermEval21_Toxic_Train.csv?token=AADCWPFIZXF6EM2QWVRICL3AUQHAE


wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter-gawron.christian/.netrc


In [3]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [4]:
%env WANDB_PROJECT=GermEval_2021

env: WANDB_PROJECT=GermEval_2021


In [5]:
%env WANDB_LOG_MODEL=true

env: WANDB_LOG_MODEL=true


In [6]:
""" Finetuning the library models for sequence classification on GermEval2021."""

import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional

import numpy as np
from datasets import load_dataset, load_metric

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers.utils import check_min_version


# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.5.0")

In [7]:
logger = logging.getLogger(__name__)

task_to_keys = {
    "germeval2021": ("comment_text", None),
}

@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.

    Using `HfArgumentParser` we can turn this class
    into argparse arguments to be able to specify them on
    the command line.
    """

    task_name: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the task to train on: " + ", ".join(task_to_keys.keys())},
    )
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached preprocessed datasets or not."}
    )
    pad_to_max_length: bool = field(
        default=True,
        metadata={
            "help": "Whether to pad all samples to `max_seq_length`. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of prediction examples to this "
            "value if set."
        },
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "A csv or a json file containing the training data."}
    )
    validation_file: Optional[str] = field(
        default=None, metadata={"help": "A csv or a json file containing the validation data."}
    )
    test_file: Optional[str] = field(default=None, metadata={"help": "A csv or a json file containing the test data."})

    def __post_init__(self):
        if self.task_name is not None:
            self.task_name = self.task_name.lower()
            if self.task_name not in task_to_keys.keys():
                raise ValueError(f"Unknown task {self.task_name}, you should pick one in " + ", ".join(task_to_keys.keys()))
        elif self.train_file is None or self.validation_file is None:
            raise ValueError("Need either a GLUE task or a training/validation file.")
        else:
            train_extension = self.train_file.split(".")[-1]
            assert train_extension in ["csv", "json"], "`train_file` should be a csv or a json file."
            validation_extension = self.validation_file.split(".")[-1]
            assert (
                validation_extension == train_extension
            ), "`validation_file` should have the same extension (csv or json) as `train_file`."

In [8]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )


## Dataset

Dataset for the [GermAval2021 Shared Task on the Identification of Toxic, Engaging, and Fact-Claiming Comments](https://germeval2021toxic.github.io/SharedTask/).

## Passing configuration

For this notebook, we use a `dict()` of configuration values.

In [9]:
def main(config):

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
    model_args, data_args, training_args = parser.parse_dict(config)

    # Detecting last checkpoint.
    last_checkpoint = None
    if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
        last_checkpoint = get_last_checkpoint(training_args.output_dir)
        if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
            raise ValueError(
                f"Output directory ({training_args.output_dir}) already exists and is not empty. "
                "Use --overwrite_output_dir to overcome."
            )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        handlers=[logging.StreamHandler(sys.stdout)],
    )
    logger.setLevel(logging.INFO if is_main_process(training_args.local_rank) else logging.WARN)

    # Log on each process the small summary:
    logger.warning(
        f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
        + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
    )
    # Set the verbosity to info of the Transformers logger (on main process only):
    if is_main_process(training_args.local_rank):
        transformers.utils.logging.set_verbosity_info()
        transformers.utils.logging.enable_default_handler()
        transformers.utils.logging.enable_explicit_format()
    logger.info(f"Training/evaluation parameters {training_args}")

    # Set seed before initializing model.
    set_seed(training_args.seed)

    # Get the datasets: you can either provide your own CSV/JSON training and evaluation files (see below)
    # or specify a GLUE benchmark task (the dataset will be downloaded automatically from the datasets Hub).
    #
    # For CSV/JSON files, this script will use as labels the column called 'label' and as pair of sentences the
    # sentences in columns called 'sentence1' and 'sentence2' if such column exists or the first two columns not named
    # label if at least two columns are provided.
    #
    # If the CSVs/JSONs contain only one non-label column, the script does single sentence classification on this
    # single column. You can easily tweak this behavior (see below)
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    if data_args.task_name == 'germeval2021':
        # Downloading and loading a dataset from the hub.
        datasets = germeval2021
    else:
        # Loading a dataset from your local files.
        # CSV/JSON training and evaluation files are needed.
        data_files = {"train": data_args.train_file, "validation": data_args.validation_file}

        # Get the test dataset: you can provide your own CSV/JSON test file (see below)
        # when you use `do_predict` without specifying a GLUE benchmark task.
        if training_args.do_predict:
            if data_args.test_file is not None:
                train_extension = data_args.train_file.split(".")[-1]
                test_extension = data_args.test_file.split(".")[-1]
                assert (
                    test_extension == train_extension
                ), "`test_file` should have the same extension (csv or json) as `train_file`."
                data_files["test"] = data_args.test_file
            else:
                raise ValueError("Need either a GLUE task or a test file for `do_predict`.")

        for key in data_files.keys():
            logger.info(f"load a local file for {key}: {data_files[key]}")

        if data_args.train_file.endswith(".csv"):
            # Loading a dataset from local csv files
            datasets = load_dataset("csv", data_files=data_files, cache_dir=model_args.cache_dir)
        else:
            # Loading a dataset from local json files
            datasets = load_dataset("json", data_files=data_files, cache_dir=model_args.cache_dir)
    # See more about loading any type of standard or custom dataset at
    # https://huggingface.co/docs/datasets/loading_datasets.html.

    # Labels
    if data_args.task_name is not None:
        is_regression = data_args.task_name == "stsb"
        if not is_regression:
            label_list = datasets["train"].features["label"].names
            num_labels = len(label_list)
        else:
            num_labels = 1
    else:
        # Trying to have good defaults here, don't hesitate to tweak to your needs.
        is_regression = datasets["train"].features["label"].dtype in ["float32", "float64"]
        if is_regression:
            num_labels = 1
        else:
            # A useful fast method:
            # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.unique
            label_list = datasets["train"].unique("label")
            label_list.sort()  # Let's sort it for determinism
            num_labels = len(label_list)

    # Load pretrained model and tokenizer
    #
    # In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.
    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        num_labels=num_labels,
        finetuning_task=data_args.task_name,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        use_fast=model_args.use_fast_tokenizer,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    model = AutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )

    # Preprocessing the datasets
    if data_args.task_name is not None:
        sentence1_key, sentence2_key = task_to_keys[data_args.task_name]
    else:
        # Again, we try to have some nice defaults but don't hesitate to tweak to your use case.
        non_label_column_names = [name for name in datasets["train"].column_names if name != "label"]
        if "sentence1" in non_label_column_names and "sentence2" in non_label_column_names:
            sentence1_key, sentence2_key = "sentence1", "sentence2"
        else:
            if len(non_label_column_names) >= 2:
                sentence1_key, sentence2_key = non_label_column_names[:2]
            else:
                sentence1_key, sentence2_key = non_label_column_names[0], None

    # Padding strategy
    if data_args.pad_to_max_length:
        padding = "max_length"
    else:
        # We will pad later, dynamically at batch creation, to the max sequence length in each batch
        padding = False

    # Some models have set the order of the labels to use, so let's make sure we do use it.
    label_to_id = None
    if (
        model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id
        and data_args.task_name is not None
        and not is_regression
    ):
        # Some have all caps in their config, some don't.
        label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
        if list(sorted(label_name_to_id.keys())) == list(sorted(label_list)):
            label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
        else:
            logger.warning(
                "Your model seems to have been trained with labels, but they don't match the dataset: ",
                f"model labels: {list(sorted(label_name_to_id.keys()))}, dataset labels: {list(sorted(label_list))}."
                "\nIgnoring the model labels as a result.",
            )
    elif data_args.task_name is None and not is_regression:
        label_to_id = {v: i for i, v in enumerate(label_list)}

    if data_args.max_seq_length > tokenizer.model_max_length:
        logger.warning(
            f"The max_seq_length passed ({data_args.max_seq_length}) is larger than the maximum length for the"
            f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
        )
    max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

    def preprocess_function(examples):
        # Tokenize the texts
        args = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

        # Map labels to IDs (not necessary for GLUE tasks)
        if label_to_id is not None and "label" in examples:
            result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
        return result

    datasets = datasets.map(preprocess_function, batched=True, load_from_cache_file=not data_args.overwrite_cache)
    if training_args.do_train:
        if "train" not in datasets:
            raise ValueError("--do_train requires a train dataset")
        train_dataset = datasets["train"]
        if data_args.max_train_samples is not None:
            train_dataset = train_dataset.select(range(data_args.max_train_samples))

    if training_args.do_eval:
        if "validation" not in datasets and "validation_matched" not in datasets:
            raise ValueError("--do_eval requires a validation dataset")
        eval_dataset = datasets["validation_matched" if data_args.task_name == "mnli" else "validation"]
        if data_args.max_eval_samples is not None:
            eval_dataset = eval_dataset.select(range(data_args.max_eval_samples))

    if training_args.do_predict or data_args.task_name is not None or data_args.test_file is not None:
        if "test" not in datasets and "test_matched" not in datasets:
            raise ValueError("--do_predict requires a test dataset")
        predict_dataset = datasets["test_matched" if data_args.task_name == "mnli" else "test"]
        if data_args.max_predict_samples is not None:
            predict_dataset = predict_dataset.select(range(data_args.max_predict_samples))

    # Log a few random samples from the training set:
    if training_args.do_train:
        for index in random.sample(range(len(train_dataset)), 3):
            logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

    # Get the metric function
    #if data_args.task_name is not None:
    # metric = load_metric("glue", data_args.task_name)
    # TODO: When datasets metrics include regular accuracy, make an else here and remove special branch from
    # compute_metrics

    # You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with a
    # predictions and label_ids field) and has to return a dictionary string to float.
    def compute_metrics(p: EvalPrediction):
        preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
        preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)
        accuracy = (preds == p.label_ids).astype(np.float32).mean().item()
        metrics = { "accuracy": accuracy }
        for val, key in enumerate(label_list):
            tp = ((preds == p.label_ids) * (preds == val)).sum().item()
            fp = ((preds != p.label_ids) * (preds == val)).sum().item()
            fn = ((preds != p.label_ids) * (preds != val)).sum().item()
       
            precision = tp / (tp + fp) if tp + fp > 0 else 0
            recall = tp / (tp + fn) if tp + fn > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
            metrics[f"precision_{key}"] = precision
            metrics[f"recall_{key}"] = recall
            metrics[f"f1_{key}"] = f1
        
        avg_pre = sum([ metrics[f"precision_{key}"] for key in label_list ]) / len(label_list)
        avg_rec = sum([ metrics[f"recall_{key}"] for key in label_list ]) / len(label_list)
        avg_f1 = 2 * avg_pre * avg_rec / (avg_pre + avg_rec)
        metrics['precision_ma'] = avg_pre
        metrics['recall_ma'] = avg_rec
        metrics['f1_ma'] = avg_f1
        return metrics

    # Data collator will default to DataCollatorWithPadding, so we change it if we already did the padding.
    if data_args.pad_to_max_length:
        data_collator = default_data_collator
    elif training_args.fp16:
        data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)
    else:
        data_collator = None

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset if training_args.do_train else None,
        eval_dataset=eval_dataset if training_args.do_eval else None,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    # Training
    if training_args.do_train:
        checkpoint = None
        if last_checkpoint is not None:
            checkpoint = last_checkpoint
        train_result = trainer.train(resume_from_checkpoint=checkpoint)
        metrics = train_result.metrics
        max_train_samples = (
            data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
        )
        metrics["train_samples"] = min(max_train_samples, len(train_dataset))

        trainer.save_model()  # Saves the tokenizer too for easy upload

        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()

    # Evaluation
    if training_args.do_eval:
        logger.info("*** Evaluate ***")

        # Loop to handle MNLI double evaluation (matched, mis-matched)
        tasks = [data_args.task_name]
        eval_datasets = [eval_dataset]
        if data_args.task_name == "mnli":
            tasks.append("mnli-mm")
            eval_datasets.append(datasets["validation_mismatched"])

        for eval_dataset, task in zip(eval_datasets, tasks):
            metrics = trainer.evaluate(eval_dataset=eval_dataset)

            max_eval_samples = (
                data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
            )
            metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

            trainer.log_metrics("eval", metrics)
            trainer.save_metrics("eval", metrics)

    if training_args.do_predict:
        logger.info("*** Predict ***")

        # Loop to handle MNLI double evaluation (matched, mis-matched)
        tasks = [data_args.task_name]
        predict_datasets = [predict_dataset]
        if data_args.task_name == "mnli":
            tasks.append("mnli-mm")
            predict_datasets.append(datasets["test_mismatched"])

        for predict_dataset, task in zip(predict_datasets, tasks):
            # Removing the `label` columns because it contains -1 and Trainer won't like that.
            predict_dataset.remove_columns_("label")
            predictions = trainer.predict(predict_dataset, metric_key_prefix="predict").predictions
            predictions = np.squeeze(predictions) if is_regression else np.argmax(predictions, axis=1)

            output_predict_file = os.path.join(training_args.output_dir, f"predict_results_{task}.txt")
            if trainer.is_world_process_zero():
                with open(output_predict_file, "w") as writer:
                    logger.info(f"***** Predict results {task} *****")
                    writer.write("index\tprediction\n")
                    for index, item in enumerate(predictions):
                        if is_regression:
                            writer.write(f"{index}\t{item:3.3f}\n")
                        else:
                            item = label_list[item]
                            writer.write(f"{index}\t{item}\n")

    return model

In [10]:
import re
import datasets
import wandb
from collections import namedtuple


COLUMNS = ['Sub1_Toxic', 'Sub2_Engaging', 'Sub3_FactClaiming']

GERMEVAL2021_RAW = datasets.Dataset.from_csv('GermEval21_Toxic_Train.csv')
for id, target in enumerate(COLUMNS):
    new_features = GERMEVAL2021_RAW.features.copy()
    new_features[target] = datasets.ClassLabel(names=['Other', target[5:]], id=id)
    GERMEVAL2021_RAW = GERMEVAL2021_RAW.cast(new_features)
    
#GERMEVAL2021_RAW.features['Sub1_Toxic']
GERMEVAL2021 = GERMEVAL2021_RAW.train_test_split(train_size=0.8)
GERMEVAL2021['validation'] = GERMEVAL2021['test']

Using custom data configuration default-e4a03502bf447b85
Reusing dataset csv (/home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-e4a03502bf447b85/0.0.0)
Loading cached processed dataset at /home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-e4a03502bf447b85/0.0.0/cache-aa151c082b74cd77.arrow
Loading cached processed dataset at /home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-e4a03502bf447b85/0.0.0/cache-5f7df1d42ac9f0bc.arrow
Loading cached processed dataset at /home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-e4a03502bf447b85/0.0.0/cache-52ac9ecece5435b8.arrow
Loading cached split indices for dataset at /home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-e4a03502bf447b85/0.0.0/cache-7653e2a15d91c773.arrow and /home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-e4a03502bf447b85/0.0.0/cache-72ba59075e2d4078.arrow


## Add 'toxic' trainig data from GermEval2018 and GermEval2019

In [11]:
toxic = [ datasets.Dataset.from_csv(file).remove_columns('Unnamed: 0') for file in ['GermEval2018_training.csv', 'GermEval2019_training.csv', 'GermEval2018_test.csv', 'GermEval2019_test.csv']]

def _map(x, y):
    return { 'comment_text': x, 'Sub1_Toxic': y}

toxic.append(GERMEVAL2021['train'].map(function=_map, input_columns=['comment_text', 'Sub1_Toxic']))

toxic_train = datasets.concatenate_datasets(toxic)

Using custom data configuration default-6687e00c163b9099
Reusing dataset csv (/home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-6687e00c163b9099/0.0.0)
Using custom data configuration default-3b288cbd53cc91cb
Reusing dataset csv (/home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-3b288cbd53cc91cb/0.0.0)
Using custom data configuration default-b4c86526510677f1
Reusing dataset csv (/home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-b4c86526510677f1/0.0.0)
Using custom data configuration default-b832f048e825c62a
Reusing dataset csv (/home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-b832f048e825c62a/0.0.0)
Loading cached processed dataset at /home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-e4a03502bf447b85/0.0.0/cache-1e7d6d6822666cbd.arrow


In [12]:
features = toxic_train.features.copy()
features['Sub1_Toxic'] = datasets.ClassLabel(names=['Other', 'Toxic'], id=id)
toxic_train = toxic_train.cast(features)

Loading cached processed dataset at /home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-6687e00c163b9099/0.0.0/cache-14ff0660a366ca4e.arrow


In [13]:
GERMEVAL2021

DatasetDict({
    train: Dataset({
        features: ['comment_id', 'comment_text', 'Sub1_Toxic', 'Sub2_Engaging', 'Sub3_FactClaiming'],
        num_rows: 2595
    })
    test: Dataset({
        features: ['comment_id', 'comment_text', 'Sub1_Toxic', 'Sub2_Engaging', 'Sub3_FactClaiming'],
        num_rows: 649
    })
    validation: Dataset({
        features: ['comment_id', 'comment_text', 'Sub1_Toxic', 'Sub2_Engaging', 'Sub3_FactClaiming'],
        num_rows: 649
    })
})

In [14]:
GERMEVAL2021['test'].features, toxic_train.features

({'comment_id': Value(dtype='int64', id=None),
  'comment_text': Value(dtype='string', id=None),
  'Sub1_Toxic': ClassLabel(num_classes=2, names=['Other', 'Toxic'], names_file=None, id=0),
  'Sub2_Engaging': ClassLabel(num_classes=2, names=['Other', 'Engaging'], names_file=None, id=1),
  'Sub3_FactClaiming': ClassLabel(num_classes=2, names=['Other', 'FactClaiming'], names_file=None, id=2)},
 {'comment_text': Value(dtype='string', id=None),
  'Sub1_Toxic': ClassLabel(num_classes=2, names=['Other', 'Toxic'], names_file=None, id=2)})

In [11]:
model_name = "german-nlp-group/electra-base-german-uncased"
#model_name = "deepset/gbert-large"
lr = 1.3e-5

model = {}

#for target in COLUMNS:
for target in ['Sub1_Toxic']:
    germeval2021 = GERMEVAL2021
    #germeval2021['train'] = toxic_train
    germeval2021 = germeval2021.rename_column(target, 'label')
    print(germeval2021["train"])

    config = {
        "model_name_or_path": model_name,
        "task_name": "GermEval2021",
        "do_train": True,
        "do_eval": True,
        "evaluation_strategy": "steps",
        "eval_steps": 100,
        #"max_seq_length": 255,
        "learning_rate": lr,
        "fp16": True,
        "num_train_epochs": 5,
        #"max_steps": 1510,
        "overwrite_output": True,
        "output_dir": f"out_{target}_{model_name}_nur_21",
        "run_name": f"run_{target}_{lr}_{model_name}"
        #"resume_from_checkpoint": 1500
    }

    model[target] = main(config)
    wandb.finish()

Dataset({
    features: ['comment_id', 'comment_text', 'label', 'Sub2_Engaging', 'Sub3_FactClaiming'],
    num_rows: 2595
})
06/09/2021 16:38:40 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
06/09/2021 16:38:40 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=out_Sub1_Toxic_german-nlp-group/electra-base-german-uncased_nur_21, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.STEPS, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=1.3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Jun09_16-38-40_jupiter, logging_strategy=IntervalStrategy.

[INFO|configuration_utils.py:490] 2021-06-09 16:38:40,554 >> loading configuration file https://huggingface.co/german-nlp-group/electra-base-german-uncased/resolve/main/config.json from cache at /home/jupyter-gawron.christian/.cache/huggingface/transformers/8aa9c8c1fb8a4b4472eed9302e63f25f5f4cc692e94603152d3f5a7c59c824ab.f988e04e2ecf3a0c9a557f8386c21c7fd145779fcebf2479a709b6e9264dc4c7
[INFO|configuration_utils.py:526] 2021-06-09 16:38:40,557 >> Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 768,
  "finetuning_task": "germeval2021",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "ge


06/09/2021 16:38:47 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at /home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-e4a03502bf447b85/0.0.0/cache-0cd7e0d38e47f6e3.arrow
06/09/2021 16:38:47 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at /home/jupyter-gawron.christian/.cache/huggingface/datasets/csv/default-e4a03502bf447b85/0.0.0/cache-0cd7e0d38e47f6e3.arrow
06/09/2021 16:38:47 - INFO - __main__ -   Sample 456 of the training set: {'Sub2_Engaging': 1, 'Sub3_FactClaiming': 1, 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'comment_id': 250, 'comment_text': '400 

[INFO|trainer.py:491] 2021-06-09 16:38:52,069 >> The following columns in the training set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: Sub2_Engaging, comment_id, Sub3_FactClaiming, comment_text.
[INFO|trainer.py:491] 2021-06-09 16:38:52,074 >> The following columns in the evaluation set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: Sub2_Engaging, comment_id, Sub3_FactClaiming, comment_text.
[INFO|trainer.py:402] 2021-06-09 16:38:52,075 >> Using amp fp16 backend
[INFO|trainer.py:1013] 2021-06-09 16:38:52,278 >> ***** Running training *****
[INFO|trainer.py:1014] 2021-06-09 16:38:52,279 >>   Num examples = 2595
[INFO|trainer.py:1015] 2021-06-09 16:38:52,281 >>   Num Epochs = 5
[INFO|trainer.py:1016] 2021-06-09 16:38:52,282 >>   Instantaneous batch size per device = 8
[INFO|trainer.py:1017] 2021-06-09 16:38:52,283 >>   Total train batch size (w. parallel, distributed &

06/09/2021 16:38:52 - ERROR - wandb.jupyter -   Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

[INFO|trainer.py:491] 2021-06-09 16:44:38,184 >> The following columns in the evaluation set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: Sub2_Engaging, comment_id, Sub3_FactClaiming, comment_text.
[INFO|trainer.py:1865] 2021-06-09 16:44:38,187 >> ***** Running Evaluation *****
[INFO|trainer.py:1866] 2021-06-09 16:44:38,189 >>   Num examples = 649
[INFO|trainer.py:1867] 2021-06-09 16:44:38,190 >>   Batch size = 8


[INFO|trainer_pt_utils.py:722] 2021-06-09 16:44:42,287 >> ***** eval metrics *****
[INFO|trainer_pt_utils.py:727] 2021-06-09 16:44:42,295 >>   epoch                     =        5.0
[INFO|trainer_pt_utils.py:727] 2021-06-09 16:44:42,297 >>   eval_accuracy             =     0.7273
[INFO|trainer_pt_utils.py:727] 2021-06-09 16:44:42,298 >>   eval_f1_Other             =     0.7968
[INFO|trainer_pt_utils.py:727] 2021-06-09 16:44:42,299 >>   eval_f1_Toxic             =     0.5855
[INFO|trainer_pt_utils.py:727] 2021-06-09 16:44:42,300 >>   eval_f1_ma                =     0.6915
[INFO|trainer_pt_utils.py:727] 2021-06-09 16:44:42,302 >>   eval_loss                 =     1.0229
[INFO|trainer_pt_utils.py:727] 2021-06-09 16:44:42,303 >>   eval_mem_cpu_alloc_delta  =        0MB
[INFO|trainer_pt_utils.py:727] 2021-06-09 16:44:42,304 >>   eval_mem_cpu_peaked_delta =        0MB
[INFO|trainer_pt_utils.py:727] 2021-06-09 16:44:42,305 >>   eval_mem_gpu_alloc_delta  =        0MB
[INFO|trainer_pt_utils.py:

wandb: Network error resolved after 0:16:44.302491, resuming normal operation.


eval/loss,1.02287
eval/accuracy,0.72727
eval/precision_Other,0.78507
eval/recall_Other,0.80886
eval/f1_Other,0.79679
eval/precision_Toxic,0.60386
eval/recall_Toxic,0.56818
eval/f1_Toxic,0.58548
eval/precision_ma,0.69447
eval/recall_ma,0.68852
eval/f1_ma,0.69148


eval/loss,▂▁▁▂▁▂▃▄▃▄▆▆▆▇███
eval/accuracy,▁▆▆▄▇▃▇▆█▇▇▇███▇▇
eval/precision_Other,▁▅▄▆▅█▅▇▇▆▅▅▆▆▆▇▆
eval/recall_Other,█▄▆▃▆▁▆▃▄▄▅▅▅▄▄▄▄
eval/f1_Other,▆▆▇▄▇▁█▅▇▆█▇▇▇▇▇▇
eval/precision_Toxic,▁▇█▇█▇█▇█▇█████▇▇
eval/recall_Toxic,▁▆▅▇▅█▅▇▇▆▅▆▆▆▆▇▇
eval/f1_Toxic,▁▇▆▇▇█▇███▇▇█████
eval/precision_ma,▁██▇█▇███████████
eval/recall_ma,▁▇▆▇▆█▆███▇▇█████
eval/f1_ma,▁▇▇▇▇█▇████▇█████


In [ ]:
(p_off, r_off, f_off, p_oth, r_oth, f_oth) = (0.808057, 0.567388, 0.666667, 0.806548, 0.930472, 0.864089)

In [ ]:
p_macro = 0.5 * (p_off + p_oth)
r_macro = 0.5 * (r_off + r_oth)
f_macro = 2 * p_macro * r_macro / (p_macro + r_macro)

In [ ]:
f_macro